<a href="https://colab.research.google.com/github/NorahAbdulaziz/VideoCaptionModel/blob/main/Features_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import shutil
import tqdm
import numpy as np
import cv2
import os
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model

In [ ]:
#Source code from:
#https://github.com/Shreyz-max/Video-Captioning/blob/main/extract_features.py
def video_to_frames(video):
    image_list=[]
    videoname=video.split('.')[0]
    path = os.path.join(path, videoname)
    
    for i in os.listdir(path):
        cv2.imread(path+"/"+i)
        image_list.append(path+"/"+i)
    return image_list

def model_cnn_load():
    model = VGG16(weights="imagenet", include_top=True, input_shape=(224, 224, 3))
    out = model.layers[-2].output
    model_final = Model(inputs=model.input, outputs=out)
    return model_final


def load_image(path):
    img = cv2.imread(path)
    img = cv2.resize(img, (224, 224))
    return img

def extract_features(video, model):
    """
    :param video: The video whose frames are to be extracted to convert into a numpy array
    :param model: the pretrained vgg16 model
    :return: numpy array of size 4096x80
    """
    video_id = video.split(".")[0]
    print(video_id)
    print(f'Processing video {video}')

    image_list = video_to_frames(video)
    samples = np.round(np.linspace(
        0, len(image_list) - 1, 40))
    image_list = [image_list[int(sample)] for sample in samples]
    images = np.zeros((len(image_list), 224, 224, 3))
    for i in range(len(image_list)):
        img = load_image(image_list[i])
        images[i] = img
    images = np.array(images)
    fc_feats = model.predict(images, batch_size=128)
    img_feats = np.array(fc_feats)
    return img_feats

def extract_feats_pretrained_cnn():
    """
    saves the numpy features from all the videos
    """
    model = model_cnn_load()
    print('Model loaded')

    if not os.path.isdir(os.path.join(path, 'feat')):
        os.mkdir(os.path.join(path, 'feat'))
    video_list = os.listdir(os.path.join(path, 'feat'))
    
    for video in video_list:
        outfile = os.path.join(path, 'feat', video.split('.')[0] + '.npy')
        img_feats = extract_features(video, model)
        np.save(outfile, img_feats)

extract_feats_pretrained_cnn()